### Example 1: Draw on a blank canvas.

In [1]:
import cv2
import numpy as np

# Create a blank image (black canvas)
# 512x512 pixels, 3 color channels (RGB), uint8 type
img = np.zeros((512, 512, 3), dtype=np.uint8)

# Draw a line: cv2.line(image, start_point, end_point, color, thickness)
cv2.line(img, (50, 50), (450, 50), (255, 0, 0), 5)  # Blue line

# Draw a rectangle: cv2.rectangle(image, top_left, bottom_right, color, thickness)
cv2.rectangle(img, (100, 100), (400, 300), (0, 255, 0), 3)  # Green rectangle

# Draw a filled rectangle (thickness = -1)
cv2.rectangle(img, (120, 120), (380, 280), (0, 255, 0), -1)

# Draw a circle: cv2.circle(image, center, radius, color, thickness)
cv2.circle(img, (256, 256), 50, (0, 0, 255), 5)  # Red circle

# Draw an ellipse: cv2.ellipse(image, center, axes, angle, startAngle, endAngle, color, thickness)
cv2.ellipse(img, (256, 256), (100, 50), 0, 0, 360, (255, 255, 0), 2)

# Draw a polygon
pts = np.array([[100, 400], [200, 350], [300, 400], [200, 450]], np.int32)
cv2.polylines(img, [pts], isClosed=True, color=(255, 255, 255), thickness=2)

# Add text
cv2.putText(img, "OpenCV Drawing", (100, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Display the image
cv2.imshow("Drawings", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Example 2: Draw on a real image.

In [2]:
# Load an image
img = cv2.imread("people.jpg")

# Draw a red rectangle around a region of interest
cv2.rectangle(img, (50, 50), (300, 200), (0, 0, 255), 2)

# Add label text
cv2.putText(img, "Object", (60, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

cv2.imshow("Annotated Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Example 3: Mouse Drawing (Interactive)

In [7]:
import cv2
import numpy as np

drawing = False  # True when mouse is pressed
ix, iy = -1, -1  # Initial coordinates

# Mouse callback function
def draw_circle(event, x, y, flags, param):
    global ix, iy, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            # Draw a circle: cv2.circle(image, center, radius, color, thickness)
            cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)

# Create a black image and window
img = np.zeros((512, 512, 3), np.uint8)
cv2.namedWindow('Paint')
cv2.setMouseCallback('Paint', draw_circle)

while True:
    cv2.imshow('Paint', img)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cv2.destroyAllWindows()


### Draw Bounding Boxes Around Detected Contours.

In [34]:
import cv2
import numpy as np

# Load or create a test image
img = np.zeros((400, 400, 3), dtype=np.uint8)
cv2.circle(img, (100, 200), 50, (255, 255, 255), -1)
cv2.rectangle(img, (200, 150), (300, 250), (255, 255, 255), -1)
cv2.line(img, (320, 100), (380, 200), (255, 255, 255), 5)

# Convert to grayscale and threshold
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Find contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#thresh: The binary image where the contours are to be found.
#   Typically, this is the result of thresholding or edge detection.​
#cv2.RETR_EXTERNAL: Only retrieves the outermost (external) contours, ignoring nested (internal) ones.​
#cv2.CHAIN_APPROX_SIMPLE: Simplifies the contour representation by compressing segments, 
#   keeping only critical points—the endpoints of lines, thus reducing memory usage.​
#contours: A list of all detected external contours, where each contour is represented by a 
#   NumPy array of boundary coordinates.​
#hierarchy: An array containing information about the image’s contour structure, 
#   such as which contours have parent/child relationships. With RETR_EXTERNAL, 
#   this hierarchy is simple, as only outer contours are considered.

# Draw contours and bounding boxes
for c in contours:
    # Contour boundary in green
    cv2.drawContours(img, [c], -1, (0, 255, 0), 2)

    # Bounding rectangle in red
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Label
    cv2.putText(img, f"({x},{y})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

cv2.imshow("Contours & Bounding Boxes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Combine Drawing + Image Processing (Colored Objects)

Here’s how to detect colored objects (say, red items) and draw bounding boxes.

In [33]:
import cv2
import numpy as np

# Load image
img = cv2.imread("..\\OpenCV Demos\\fruits.jpg")

# Convert to HSV color space
# hsv: Hue (BGR), Saturation (purity), Value (brightness/intensity).
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define color range for red (two ranges for red hue)
lower_red1 = np.array([0, 120, 70])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 120, 70])
upper_red2 = np.array([180, 255, 255])

# Mask for red color
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
mask = cv2.bitwise_or(mask1, mask2)

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes
i = 0
for c in contours:
    global i
    #c is a contour, which is basically a NumPy array of points (x, y coordinates) outlining a detected shape or object.
    #cv2.contourArea() is an OpenCV function that calculates the area enclosed by that contour.
    #So, this line computes the number of pixels inside the contour boundary — effectively the size of the shape in the image.
    area = cv2.contourArea(c)
    if area > 500:
        i += 1
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, f"Red Object {i}:{area}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

cv2.imshow("Detected Red Objects", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load an image (use the shapes.png you have)
img = cv2.imread("..\\OpenCV Demos\\fruits.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Convert to binary image
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Find contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Loop through each contour
for i, c in enumerate(contours):
    area = cv2.contourArea(c)
    print(f"Contour {i+1}: Area = {area}")

    # Draw the contour and label the area on the image
    cv2.drawContours(img, [c], -1, (0, 255, 0), 2)
    x, y, w, h = cv2.boundingRect(c)
    #cv2.putText(img, f"{int(area)}", (x, y - 10),
    #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

cv2.imshow("Contours with Areas", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Blend overlay with image.

In [42]:
overlay = img.copy()
cv2.rectangle(overlay, (50, 50), (250, 200), (0, 255, 0), -1)
alpha = 0.4  # transparency factor

# Blend overlay with image
img_new = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
cv2.imshow("Blend Overlay", img_new)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 🧩 Step 1 — Draw Rectangle with Mouse (ROI - Region of Interest Selection)

In [46]:
import cv2

# Load image
img = cv2.imread("people.jpg")
clone = img.copy()

drawing = False
ix, iy = -1, -1

# Mouse callback function
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            temp = clone.copy()
            cv2.rectangle(temp, (ix, iy), (x, y), (0, 255, 0), 2)
            cv2.imshow("Draw ROI", temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 2)
        cv2.imshow("Draw ROI", img)
        print(f"ROI selected: Top-left ({ix},{iy}), Bottom-right ({x},{y})")

cv2.namedWindow("Draw ROI")
cv2.setMouseCallback("Draw ROI", draw_rectangle)

cv2.imshow("Draw ROI", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


ROI selected: Top-left (231,368), Bottom-right (641,216)
ROI selected: Top-left (672,316), Bottom-right (672,316)
ROI selected: Top-left (684,321), Bottom-right (183,189)
ROI selected: Top-left (336,95), Bottom-right (465,227)
ROI selected: Top-left (657,253), Bottom-right (327,5)


### 🧩 Step 2 — Crop the Selected ROI Automatically

We can extend the previous example to crop and show the selected region after drawing:

In [47]:
import cv2

img = cv2.imread("people.jpg")
clone = img.copy()
drawing = False
ix, iy = -1, -1

def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        temp = clone.copy()
        cv2.rectangle(temp, (ix, iy), (x, y), (255, 0, 0), 2)
        cv2.imshow("Select ROI", temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x, y), (255, 0, 0), 2)
        cv2.imshow("Select ROI", img)
        x1, y1 = min(ix, x), min(iy, y)
        x2, y2 = max(ix, x), max(iy, y)
        roi = clone[y1:y2, x1:x2]
        if roi.size > 0:
            cv2.imshow("Cropped ROI", roi)

cv2.namedWindow("Select ROI")
cv2.setMouseCallback("Select ROI", draw_rectangle)

cv2.imshow("Select ROI", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 🧩 Step 3 — Multi-Object Annotation Tool

Let’s make it record all rectangles you draw (for dataset labeling):

In [48]:
import cv2

img = cv2.imread("people.jpg")
clone = img.copy()
rectangles = []
drawing = False
ix, iy = -1, -1

def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        temp = clone.copy()
        for rect in rectangles:
            cv2.rectangle(temp, rect[0], rect[1], (0, 255, 0), 2)
        cv2.rectangle(temp, (ix, iy), (x, y), (0, 0, 255), 2)
        cv2.imshow("Annotate", temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 2)
        rectangles.append(((ix, iy), (x, y)))
        print(f"Added ROI: {rectangles[-1]}")
        cv2.imshow("Annotate", img)

cv2.namedWindow("Annotate")
cv2.setMouseCallback("Annotate", draw_rectangle)

cv2.imshow("Annotate", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Final ROIs:", rectangles)


Added ROI: ((168, 33), (238, 114))
Added ROI: ((322, 110), (385, 157))
Added ROI: ((441, 144), (476, 189))
Added ROI: ((549, 172), (594, 214))
Final ROIs: [((168, 33), (238, 114)), ((322, 110), (385, 157)), ((441, 144), (476, 189)), ((549, 172), (594, 214))]
